# Import Packages

In [1]:
import numpy as np
import math
import time
from multiprocessing import Pool

# go up two directories and import code packages
import os, sys
# code_dir=os.path.dirname(os.path.dirname(os.getcwd()))
# sys.path.append(code_dir)
# os.chdir(code_dir)

from LBL_funcs_fullSpectrum import *
from LBL_funcs_shortwave import *
from LBL_funcs_inclined import *

import warnings
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'LBL_funcs_shortwave'

# Special run 1: Grid convergence test, standalone code

In [ ]:
## general inputs
N_layer= 18 # the number of atmospheric layers
N_bundles=1000 # the number of photon bundles per wavenumber
dnu = 10 # spectral resolution 0.1 is enough, 0.01 is too fine, especially for cloudy periods
nu=np.arange(2500,35000,dnu) # spectral grid on wavenumber
molecules=['H2O','CO2','O3','N2O','CH4','O2','N2'] # considered atmospheric gases
#current trace gas surface vmr from http://cdiac.ornl.gov/pns/current_ghg.html, except O3
vmr0={'H2O':0.03,'CO2':399.5/10**6,'O3':50/10**9,'N2O':328/10**9,
          'CH4':1834/10**9,'O2':2.09/10,'N2':7.81/10}
model='AFGL midlatitude summer' #profile model, 'AFGL tropical','AFGL midlatitude summer','AFGL midlatitude winter',
#'AFGL subarctic summer','AFGL subarctic winter','AFGL US standard'
cld_model = 'default' # cloud model, 'default' or 'caseX'
period = 'day' # choose 'day' or 'night' for proper temperature profile
spectral='SW' # choose 'LW' or 'SW'
alt = 0 # altitude of location, by default is 0

##inputs for desired atmoshperic and surface conditions
#surface_v=['case2','PV','CSP'] # name of surface
surface_v=['case2'] # name of surface
rh0_v=np.arange(70,75,5)/100 # surface relative humidity
T_surf_v=np.arange(294.2,295,5) # surface temperature
AOD_v=np.array([0.1]) # aerosol optical depth at 479.5 nm
COD_v=np.array([0.0]) # cloud optical depth at 479.5 nm
#CODs=np.array([0,0.1,0.3,0.5,0.7,1.0,3.0,5.0,10.0,50.0]) # cloud optical depth at 479.5 nm
kap_v=[[4,5,6]]
#kap_v=[[10],[8,9,10],[6,7,8,9,10],[4,5,6,7,8,9,10],
#      [22],[19,20,21,22],[16,17,18,19,20,21,22],[13,14,15,16,17,18,19,20,21,22],
#      [10,11,12,13,14,15,16,17,18,19,20,21,22],[7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22],
#      [4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22]] # cloud residancy layer

##inputs of angles
#th0_v = np.array([0,15,30,45,60,75])
th0_v = np.array([30])
theta0_v = th0_v / 180 * math.pi  # solar zenith angle in rad
phi0 = 0 / 180 * math.pi  #solar azimuth angle in rad
del_angle= 0.5/180*math.pi # DNI acceptance angle, in rad, default is 0.5 degree
beta_v=np.array([0,15,30,45,60,75,90])/180*math.pi # surface tilt angles in rad
phi_v=phi0+np.array([0,45,90,135,180])/180*math.pi # surface azimuth angles in rad
isTilted=False # whether to compute transposition on inclined surfaces

##inputs of finite power plant computation
x0_v=120.0*np.tan(theta0_v)*np.cos(phi0) # photon starting x location, in km
y0_v=120.0*np.tan(theta0_v)*np.sin(phi0) # photon starting y location, in km
R_pp=1 # radius of power plant in km
is_pp=False # whether to consider power plant of finite size
#dx_v=np.arange(-5.0,5.2,0.2)# displacement of input photon location
dx_v=np.array([0.0])# displacement of input photon location
## folder directory to store the results
file_dir='results_shortwave/grid/' # create the directory first

In [ ]:
N_layer_v=[18,36,54,72]
dnu_v=[1,2,3,5,10]
N_bundles_v=[100,500,1000,5000]
# compute case by case
for iNl in range(0,len(N_layer_v)):
    for iDnu in range(0,len(dnu_v)):
        nu=np.arange(2500,35000,dnu_v[iDnu])
        for iNb in range(0,len(N_bundles_v)):
            inputs_main={'N_layer':N_layer_v[iNl], 'N_bundles':N_bundles_v[iNb], 'nu':nu, 
                         'molecules':molecules,'vmr0':vmr0,'model':model,'cld_model':cld_model,
                         'period':period,'spectral':spectral,'surface':surface_v[0], 'alt':alt}
            properties={'rh0':rh0_v[0],'T_surf':T_surf_v[0],'AOD':AOD_v[0],
                        'COD':COD_v[0],'kap':kap_v[0]}
            angles={'theta0':theta0_v[0],'phi0':phi0,'del_angle':del_angle,'beta':beta_v,
                    'phi':phi_v,'isTilted':isTilted}
            finitePP={'x0':-x0_v[0]+dx_v[0],'y0':-y0_v[0],'R_pp':R_pp,'is_pp':is_pp}                          
            print ("Start MonteCarlo once.")
            start_time = time.time()
            out = LBL_shortwave(properties,inputs_main,angles,finitePP)
            fileName1="Results_{}layers_{}bundles_dnu={}".format(
                N_layer_v[iNl],N_bundles_v[iNb],dnu_v[iDnu])
            np.save(file_dir+fileName1,out)# save results to local directory
            print ("End MonteCarlo once.")
            end_time = time.time()
            print ("CPU time:", end_time - start_time)

# Special run 2. CIRC cases, standalone code

In [2]:
!pwd

/Users/dengnan/Documents/git_store/Shortwave_MCRTM


In [3]:
model='case1' #*************change model
cld_model='case1' #*************change model
spectral='SW'

file1="data/CIRC/"+model+"_input&output/cloud_input_"+model+".txt" 
data_cld=np.genfromtxt(file1,skip_header=2)
CF=data_cld[:,1]
CF[np.isnan(CF)]=0
kap,=np.nonzero(CF) # get kap from files

file="data/CIRC/"+model+"_input&output/Tsfc_sza_nlev_"+model+".txt" 
data_gen=np.genfromtxt(file, delimiter=',') # get t0, sza, nlev
N_layer=int(data_gen[5]-1)
t0=data_gen[6]
theta0=data_gen[7]/180*math.pi
F_os_tot=data_gen[8]

# ****************spectral resolution on wavelength or wavenubmer basis*********
dnu=3.00
nu_min=2500
#nu_max=35000 #******* > 10^4/0.29 um
nu_max=40000 #******* > 10^4/0.29 um
nu=np.arange(nu_min,nu_max,dnu)
lam=1e4/nu

molecules=['H2O','CO2','O3','N2O','CO','CH4','O2'] # add CO
#inputs=[N_layer,model,molecules,nu,cld_model,spectral]

# get p,z,t profile
file="data/CIRC/"+model+"_input&output/level_input_"+model+".txt" 
data=np.genfromtxt(file,skip_header=1)
data=np.vstack((data[0,:],data))
z=data[:,1]*1e3 # in m
p=data[:,2]*1e2 # in Pa
t=data[:,3] # in K
za=np.zeros(len(z)-1)
for i in range(0,N_layer+1):
    za[i]=(z[i+1]+z[i])/2
# get layer averaged pa, ta
file="data/CIRC/"+model+"_input&output/layer_input_"+model+".txt" 
data=np.genfromtxt(file,skip_header=3)
data=np.vstack((data[0,:],data))
pa=data[:,1]*1e2 # in Pa
ta=data[:,2]
t[0]=t[1]
ta[0]=ta[1]
# set densities of each gas
vmr,densities=set_vmr_circ(model,molecules,pa,ta)
# get surface albedo, F_dw_os and etc...
file="data/CIRC/"+model+"_input&output/sfcalbedo_input_"+model+".txt" 
data=np.genfromtxt(file,skip_header=6)
F_dw_os=np.interp(nu,data[:,0],data[:,3])
ratio=F_os_tot/np.cos(theta0)/np.trapz(F_dw_os*1e-3,nu) # added 07/23/2018
F_dw_os*=np.abs(ratio)
alpha_s=1.0-np.interp(nu,data[:,0],data[:,1]) #***************changed!!!!

In [ ]:
N_bundles=1000 #************
surface=model
finitePP={'x0':0.0,'y0':0.0,'R_pp':1.0,'is_pp':False}  
angles_cor={'theta0':theta0,'phi0':30/180*math.pi,'del_angle':0.5/180*math.pi,'beta':[],
                    'phi':[],'isTilted':False}
inputs_main={'N_layer':N_layer, 'N_bundles':N_bundles, 'nu':nu, 
             'molecules':molecules,'vmr0':[],'model':model,'cld_model':cld_model,
             'period':'day','spectral':'SW','surface':surface, 'alt':0} 

## running script for SW Monte Carlo simulation
# start the flux calculation
AOD=1.0#*********** for CIRC cases
COD=1.0#********* for CIRC cases
if not kap.any():
    COD=0.0#***************for CIRC clear cases
print (COD) 

print ("Start getMixKappa.")
# get na using ideal gas law
NA=6.022140857*1e23
na2=pa*NA/8.314/ta/10**6 # in unit of /cm3
coeff_gas,coeff_aer,coeff_cld,coeff_all=getMixKappa(inputs_main,densities,pa,ta,z,za,na2,AOD,COD,kap)
print ("Finish getMixKappa.")
# get aerosol profiles of CIRC cases
coeff_aer=aerosol_circ(model,lam,z)
# change overall coefficients
coeff_all[0]=coeff_gas[0]+coeff_aer[0]+coeff_cld[0]
coeff_all[1]=coeff_gas[1]+coeff_aer[1]+coeff_cld[1]

In [ ]:
# partial code from function LBL_shortwave
ke_M=coeff_all[0]+coeff_all[1]
rho_mix_M=coeff_all[1]/ke_M
rho_mix_M[np.isnan(rho_mix_M)]=0
sca_gas_M=coeff_gas[1]/coeff_all[1]
sca_gas_M[np.isnan(sca_gas_M)]=0
sca_aer_M=(coeff_gas[1]+coeff_aer[1])/coeff_all[1]
sca_aer_M[np.isnan(sca_aer_M)]=0
g_aer_M=coeff_aer[2]
g_cld_M=coeff_cld[2]

z_V=z*100.0 # change unit to cm, so z_V*ke has unit of 1
z_V=np.hstack(z_V)
N_lam=len(nu)
# prepare inputs for parallel computing
list_args = []
for k in range(0, (int)(N_lam)):
    if (surface=='CSP'):
        temp=alpha_s[k,:]
    else:
        temp=alpha_s[k]
    # inputs is a python dict object
    inputs={'nu':nu[k],'N_layer':N_layer,'z_V':z_V,'surface':surface,'alpha_s':temp,'alpha_s_g':temp,
    'ke':ke_M[:,k],'rho_mix':rho_mix_M[:,k],'sca_gas':sca_gas_M[:,k],'sca_aer':sca_aer_M[:,k],
    'g_aer':g_aer_M[:,k],'g_c':g_cld_M[:,k]}
    args = [N_bundles,inputs,angles_cor,F_dw_os[k],finitePP]
    list_args.append(args)
# iterate line-by-line and bundle-by-bundle, parallel
pool=Pool()
results = list(pool.map(MonteCarlo_mono, list_args))
pool.terminate()

# process results to output dni, ghi, dhi, and irradiance on inclined surfaces
n_uw_M,n_dw_M,n_gas_M= [np.zeros((N_layer + 2,N_lam)) for i in range(0, 3)] # total across boundaries
uw_rxyz_M, dw_rxyz_M, uw_xyz_M, dw_xyz_M = [[] for i in range(0,4)] # for transposition and finite power plant cases
dw_rx,dw_ry,dw_rz,uw_rx,uw_ry,uw_rz=[np.zeros((N_bundles*2,N_lam))*np.nan for i in range(0,6)]
dw_x,dw_y,dw_z,uw_x,uw_y,uw_z=[np.zeros((N_bundles*2,N_lam))*np.nan for i in range(0,6)] # track photon location

for k in range(0, (int)(N_lam)):  # line-by-line
    output=results[k]
    n_uw_M[:, k] = output['n_uw']
    n_dw_M[:, k] = output['n_dw']
    n_gas_M[:,k] = output['n_gas']

    uw_rxyz=output['uw_rxyz']
    dw_rxyz=output['dw_rxyz']
    uw_xyz=output['uw_xyz']
    dw_xyz=output['dw_xyz']
    N_uw=len(uw_rxyz)
    N_dw=len(dw_rxyz)

    uw_rx[0:N_uw,k]=np.array([x[0] for x in uw_rxyz])
    uw_ry[0:N_uw,k]=np.array([x[1] for x in uw_rxyz])
    uw_rz[0:N_uw,k]=np.array([x[2] for x in uw_rxyz])
    dw_rx[0:N_dw,k]=np.array([x[0] for x in dw_rxyz])
    dw_ry[0:N_dw,k]=np.array([x[1] for x in dw_rxyz])
    dw_rz[0:N_dw,k]=np.array([x[2] for x in dw_rxyz])

    dw_x[0:N_dw,k]=np.array([x[0] for x in dw_xyz])
    dw_y[0:N_dw,k]=np.array([x[1] for x in dw_xyz])
    dw_z[0:N_dw,k]=np.array([x[2] for x in dw_xyz])

ratio=F_dw_os*np.cos(theta0)/(N_bundles*1.0)
# get transposition results
out=MCtransposition(uw_rx, uw_ry, uw_rz, dw_rx, dw_ry, dw_rz, angles_cor,ratio)
out2={'F_dw':n_dw_M*ratio,'F_uw':n_uw_M*ratio, 'F_gas':n_gas_M*ratio,
      'F_dni':out['F_dni'],'F_dhi':out['F_dhi'],'F_ghi_2D':[],
      'F_inclined':out['F_inclined'],'F_dcs':out['F_dcs']}

fileName="results_shortwave/CIRC/"+model+"_HG_final"#************** change for different scenarios
np.save(fileName,out2)

In [ ]:
# quick check the results
data=out2
Fdw_tot=data['F_dw'][1,:]
Fuw_toa=data['F_uw'][-1,:]
Fuw_sur=data['F_uw'][1,:]
print(np.trapz(Fdw_tot/1e3,nu))
print(np.trapz(Fuw_toa/1e3,nu))
print(np.trapz(Fuw_sur/1e3,nu))

# Special run 3. Transposition cases, standalone code

In [ ]:
## general inputs
N_layer= 54 # the number of atmospheric layers
N_bundles=1000 # the number of photon bundles per wavenumber
dnu = 3 # spectral resolution 0.1 is enough, 0.01 is too fine, especially for cloudy periods
nu=np.arange(2500,35000,dnu) # spectral grid on wavenumber
molecules=['H2O','CO2','O3','N2O','CH4','O2','N2'] # considered atmospheric gases
#current trace gas surface vmr from http://cdiac.ornl.gov/pns/current_ghg.html, except O3
vmr0={'H2O':0.03,'CO2':399.5/10**6,'O3':50/10**9,'N2O':328/10**9,
          'CH4':1834/10**9,'O2':2.09/10,'N2':7.81/10}
model='AFGL midlatitude summer' #profile model, 'AFGL tropical','AFGL midlatitude summer','AFGL midlatitude winter',
#'AFGL subarctic summer','AFGL subarctic winter','AFGL US standard'
cld_model = 'default' # cloud model, 'default' or 'caseX'
period = 'day' # choose 'day' or 'night' for proper temperature profile
spectral='SW' # choose 'LW' or 'SW'
alt = 0 # altitude of location, by default is 0

##inputs for desired atmoshperic and surface conditions
#surface_v=['case2','PV','CSP'] # name of surface
surface_v=['case2'] # name of surface
rh0_v=np.arange(70,75,5)/100 # surface relative humidity
T_surf_v=np.arange(294.2,295,5) # surface temperature
AOD_v=np.array([0.05]) # aerosol optical depth at 479.5 nm
COD_v=np.array([0.0,0.5,1.0,5.0,10.0]) # cloud optical depth at 479.5 nm
kap_v=[[4,5,6]]

##inputs of angles
th0_v = np.array([30])
theta0_v = th0_v / 180 * math.pi  # solar zenith angle in rad
phi0 = 0 / 180 * math.pi  #solar azimuth angle in rad
del_angle= 0.5/180*math.pi # DNI acceptance angle, in rad, default is 0.5 degree
beta_v=np.array([0,15,30,45,60,75,90])/180*math.pi # surface tilt angles in rad
phi_v=phi0+np.array([0,45,90,135,180])/180*math.pi # surface azimuth angles in rad
isTilted=True # whether to compute transposition on inclined surfaces

##inputs of finite power plant computation
x0_v=120.0*np.tan(theta0_v)*np.cos(phi0) # photon starting x location, in km
y0_v=120.0*np.tan(theta0_v)*np.sin(phi0) # photon starting y location, in km
R_pp=1 # radius of power plant in km
is_pp=False # whether to consider power plant of finite size
dx_v=np.array([0.0])# displacement of input photon location
## folder directory to store the results
file_dir='results_shortwave/inclined/' # create the directory first

In [ ]:
# compute case by case
inputs_main={'N_layer':N_layer, 'N_bundles':N_bundles, 'nu':nu, 'molecules':molecules,'vmr0':vmr0,
   'model':model,'cld_model':cld_model,'period':period,'spectral':spectral,'surface':surface_v[0], 
             'alt':alt}
angles={'theta0':theta0_v[0],'phi0':phi0,'del_angle':del_angle,'beta':beta_v,
        'phi':phi_v,'isTilted':isTilted}
finitePP={'x0':-x0_v[0]+dx_v[0],'y0':-y0_v[0],'R_pp':R_pp,'is_pp':is_pp}  
for iCOD in range(0,len(COD_v)):
    properties={'rh0':rh0_v[0],'T_surf':T_surf_v[0],'AOD':AOD_v[0],
                'COD':COD_v[iCOD],'kap':kap_v[0]}                    
    print ("Start MonteCarlo once.")
    start_time = time.time()
    out = LBL_shortwave(properties,inputs_main,angles,finitePP)
    fileName1="Results_{}_RH={}_AOD={}_COD={}".format(
        surface_v[0],rh0_v[0],AOD_v[0],COD_v[iCOD])
    np.save(file_dir+fileName1,out)# save results to local directory
    print ("End MonteCarlo once.")
    end_time = time.time()
    print ("CPU time:", end_time - start_time)